### Проверка ДЗ 1

In [90]:
import json

In [91]:
# Указываем номер варианта проверяемого
variant = 16

# Считываем исходные данные
with open(f"homework_1/input_data/{variant}.json") as f:
    input_data = json.load(f)

In [92]:
# Предварительно лучше перейти в ветку которая на PR, чтобы файл с ответом уже был в папке и его не скачивать отдельно
# Как это сделать: https://codex.so/git-checkout-on-pull-request-from-fork

# Путь к файлу с ответом ученика
output_path = "homework_1/muravtsev/output.json"

# Считываем выходные данные
with open(output_path) as f:
    output_data = json.load(f)

In [93]:
input_data

{'gamma_water': 1.0531826688400516,
 'md_vdp': 2795.4461926851586,
 'd_tub': 0.6292958170855484,
 'angle': 48.26556535739659,
 'roughness': 0.00023828101188969189,
 'p_wh': 151.89057740712013,
 't_wh': 26.056301710698456,
 'temp_grad': 2.211303276181458}

In [94]:
from math import sin, pi

In [95]:
input_data["md_vdp"]*sin(input_data["angle"]*pi/180)

2086.0688620165038

In [96]:
# Получаем эталонный ответ с помощью uniflocpy
import pandas as pd
import unifloc.tools.units_converter as uc
import unifloc.common.trajectory as traj
import unifloc.common.ambient_temperature_distribution as amb
import unifloc.pipe.pipeline as pipel
import unifloc.pvt.fluid_flow as fl

# Инициализация исходных данных класса Pipeline
trajectory = traj.Trajectory(pd.DataFrame(columns=["MD", "TVD"],
                                     data=[[0, 0],
                                           [input_data["md_vdp"], input_data["md_vdp"]*sin(input_data["angle"]*pi/180)]]))

ambient_temperature_data = {"MD": [0, input_data["md_vdp"]], "T": [input_data["t_wh"]+273.15, 
                                                                   input_data["t_wh"]+273.15 + 2.71 / 100 * input_data["md_vdp"]*sin(input_data["angle"]*pi/180)]}

amb_temp = amb.AmbientTemperatureDistribution(ambient_temperature_data)

fluid_data = {
    "q_fluid": uc.convert_rate(100, "m3/day", "m3/s"),
    "pvt_model_data": {
        "black_oil": {
            "gamma_gas": 0.7, "gamma_wat": input_data["gamma_water"], "gamma_oil": 0.8,
            "wct": 1, "phase_ratio": {"type": "GOR", "value": 401}}}}

fluid = fl.FluidFlow(**fluid_data)

d = input_data["d_tub"] / 10
top_depth = 0
bottom_depth = input_data["md_vdp"]
roughness = input_data["roughness"]
p_wh = input_data["p_wh"] * 101325  # Па
flow_direction = -1
# Инициализация объекта pvt-модели
pipe = pipel.Pipeline(top_depth, bottom_depth, d, roughness,
                      trajectory, fluid, amb_temp)

# Расчёт VLP
q_liq_array = output_data["q_liq"]
p_wf_array = []
for q_liq in q_liq_array:
    p_wf, *_ = pipe.calc_pt("top", p_wh, flow_direction, q_liq/86400, 1)
    p_wf_array.append(p_wf/101325)

In [98]:
# Сравним 2 массива
import numpy.testing

# Допустимый процент ошибки
rtol = 0.02
try:
    numpy.testing.assert_allclose(p_wf_array, output_data["p_wf"], rtol=rtol)
except AssertionError as e:
    
    print("Ответ студента неверный")
    
    print(e)

### Проверка ДЗ 2

In [1]:
import json

In [4]:
# Указываем номер варианта проверяемого
variant = 6

# Считываем исходные данные
with open(f"homework_2/input_data/{variant}-1.json") as f:
    input_data_1 = json.load(f)
    
with open(f"homework_2/input_data/{variant}-2.json") as f:
    input_data_2 = json.load(f)

In [5]:
# Предварительно лучше перейти в ветку которая на PR, чтобы файл с ответом уже был в папке и его не скачивать отдельно
# Как это сделать: https://codex.so/git-checkout-on-pull-request-from-fork

# Путь к файлу с ответом ученика
output_path = "homework_2/gudilov/output.json"

# Считываем выходные данные
with open(output_path) as f:
    output_data = json.load(f)

In [10]:
input_data_1

{'fluid': {'gamma_oil': 0.851724456241123,
  'gamma_water': 1.0502998368310799,
  'gamma_gas': 0.7304998513017404,
  'wct': 0.2720726926929928,
  'rp': 294.6164789804204},
 'pipe': {'tubing': {'d': 0.08921736121695023,
   'md': 1400,
   'roughness': 0.00045733210160047854},
  'casing': {'d': 0.11818220557958338, 'roughness': 0.0006044111356378995}},
 'inclinometry': {'md': [0,
   1400,
   1850.0098193139966,
   1942.7396915429672,
   2002.404007300809,
   2055.9257655594815,
   2062.591682402726,
   2126.704108334986],
  'tvd': [0,
   1300,
   1576.9299136601012,
   1635.5589137764491,
   1648.3012952980816,
   1686.4388932333995,
   1689.4105356982857,
   1728.5095221986921]},
 'temperature': {'t_res': 65.03439498049232, 'temp_grad': 2.720873261686019},
 'reservoir': {'md_vdp': 2126.704108334986,
  'p_res': 205.90147843835393,
  'pi': 0.5870919774794668},
 'regime': {'p_wh': 27.099308518811014}}

In [7]:
input_data_2

{'md_valve': 1148.3144749454036}

In [8]:
from unifloc.well.gaslift_well import GasLiftWell

In [9]:
GasLiftWell(a)

unifloc.well.gaslift_well.GasLiftWell

#### Задание 1

In [82]:
# Получаем эталонный ответ с помощью uniflocpy
import pandas as pd
import numpy as np
import unifloc.tools.units_converter as uc
from unifloc.well.gaslift_well import GasLiftWell

df = {"MD": input_data_1["inclinometry"]["md"],
      "TVD": input_data_1["inclinometry"]["tvd"]}

fluid_data = {"q_fluid": uc.convert_rate(100, "m3/day", "m3/s"),
              "pvt_model_data": 
              {"black_oil": 
               {"gamma_gas": input_data_1["fluid"]["gamma_gas"], 
                "gamma_wat": input_data_1["fluid"]["gamma_water"], 
                "gamma_oil": input_data_1["fluid"]["gamma_oil"],
                "wct": input_data_1["fluid"]["wct"], 
                "phase_ratio": {"type": "GOR", "value": input_data_1["fluid"]["rp"]}}}}

pipe_data = {"casing": {"bottom_depth": input_data_1["reservoir"]["md_vdp"],
                        "d": input_data_1["pipe"]["casing"]["d"],
                        "roughness": input_data_1["pipe"]["casing"]["roughness"]},
             "tubing": {"bottom_depth": input_data_1["pipe"]["tubing"]["md"],
                        "d": input_data_1["pipe"]["tubing"]["d"],
                        "roughness": input_data_1["pipe"]["tubing"]["roughness"]}}
well_trajectory_data = {"inclinometry": df}
equipment_data = None
ambient_temperature_data = {"MD": [0, input_data_1["reservoir"]["md_vdp"]],
                            "T": [input_data_1["temperature"]["t_res"] + 273.15 - input_data_1["temperature"]["temp_grad"]/ 100 * input_data_1["inclinometry"]["tvd"][-1],
                                  input_data_1["temperature"]["t_res"] + 273.15]}

# Инициализация объекта скважины
well = GasLiftWell(fluid_data, pipe_data, well_trajectory_data,
                   ambient_temperature_data, equipment_data)

def _min_func(q_liq, p_res, pi, p_wh, wct):
    # Расчёт забойного от пластового
    p_wf_res = p_res - q_liq / pi
    
    # Расчет забойного от буферного
    p_wf_well = well.calc_pwf_pfl(p_wh * 101325,
                             q_liq / 86400,
                             wct)
    return abs(p_wf_res - p_wf_well / 101325)

from scipy.optimize import minimize_scalar

q_liq = minimize_scalar(_min_func,
                        bounds=(1, input_data_1["reservoir"]["p_res"] * input_data_1["reservoir"]["pi"]),
                        args=(
                            input_data_1["reservoir"]["p_res"],
                            input_data_1["reservoir"]["pi"],
                            input_data_1["regime"]["p_wh"],
                            input_data_1["fluid"]["wct"]
                                                         ),
                       method="bounded")
p_wf = input_data_1["reservoir"]["p_res"] - q_liq.x / input_data_1["reservoir"]["pi"]

In [72]:
q_liq

     fun: 1.693173430794559e-06
 message: 'Solution found.'
    nfev: 25
  status: 0
 success: True
       x: 28.361220058532496

In [74]:
p_wf

157.5935111241107

In [81]:
# Сравним ответ на 1 задание
import pytest

# Допустимый процент ошибки
rtol = 0.02

try:
    assert p_wf == pytest.approx(output_data["t1"]["pwf"], rtol)
except AssertionError as e:
    print("Ответ студента неверный")
    print("Unifloc: ", p_wf)
    print("Студент: ", output_data["t1"]["pwf"])

#### Задание 2

In [91]:
# Получаем эталонный ответ с помощью uniflocpy
import pandas as pd
import numpy as np
import unifloc.tools.units_converter as uc
from unifloc.well.gaslift_well import GasLiftWell

df = {"MD": input_data_1["inclinometry"]["md"],
      "TVD": input_data_1["inclinometry"]["tvd"]}

fluid_data = {"q_fluid": uc.convert_rate(100, "m3/day", "m3/s"),
              "pvt_model_data": 
              {"black_oil": 
               {"gamma_gas": input_data_1["fluid"]["gamma_gas"], 
                "gamma_wat": input_data_1["fluid"]["gamma_water"], 
                "gamma_oil": input_data_1["fluid"]["gamma_oil"],
                "wct": input_data_1["fluid"]["wct"], 
                "phase_ratio": {"type": "GOR", "value": input_data_1["fluid"]["rp"]}}}}

pipe_data = {"casing": {"bottom_depth": input_data_1["reservoir"]["md_vdp"],
                        "d": input_data_1["pipe"]["casing"]["d"],
                        "roughness": input_data_1["pipe"]["casing"]["roughness"]},
             "tubing": {"bottom_depth": input_data_1["pipe"]["tubing"]["md"],
                        "d": input_data_1["pipe"]["tubing"]["d"],
                        "roughness": input_data_1["pipe"]["tubing"]["roughness"]}}
well_trajectory_data = {"inclinometry": df}
equipment_data = {"gl_system": {
                  "valve1": {"h_mes": input_data_2["md_valve"], "d": 0.006,
                              "p_valve": uc.convert_pressure(3, "atm", "Pa"),
                             "valve_type": "ЦКсОК"}
                                              }
                  }
ambient_temperature_data = {"MD": [0, input_data_1["reservoir"]["md_vdp"]],
                            "T": [input_data_1["temperature"]["t_res"] + 273.15 - input_data_1["temperature"]["temp_grad"]/ 100 * input_data_1["inclinometry"]["tvd"][-1],
                                  input_data_1["temperature"]["t_res"] + 273.15]}

# Инициализация объекта скважины
well = GasLiftWell(fluid_data, pipe_data, well_trajectory_data,
                   ambient_temperature_data, equipment_data)


def _min_func(q_liq, p_res, pi, p_wh, wct, q_inj):
    # Расчёт забойного от пластового
    p_wf_res = p_res - q_liq / pi
    
    # Расчет забойного от буферного
    p_wf_well = well.calc_pwf_pfl(p_wh * 101325,
                             q_liq / 86400,
                             wct, q_gas_inj=q_inj/86400)
    return abs(p_wf_res - p_wf_well / 101325)
from scipy.optimize import minimize_scalar


# Создадим массив закачек
# Чтобы быстрее проверять - возьмем ответ и посчитаем в его окрестности
min_val = max(0, output_data["t2"]["q_inj"] - 10000)
max_val = output_data["t2"]["q_inj"] + 10000

q_inj_array = np.linspace(min_val, max_val, 20)
q_liq_array = []

for q_inj in q_inj_array:
    q_liq = minimize_scalar(_min_func,
                        bounds=(1, input_data_1["reservoir"]["p_res"] * input_data_1["reservoir"]["pi"]),
                        args=(
                            input_data_1["reservoir"]["p_res"],
                            input_data_1["reservoir"]["pi"],
                            input_data_1["regime"]["p_wh"],
                            input_data_1["fluid"]["wct"],
                            q_inj
                                                         ),
                       method="bounded")
    if q_liq.fun > 100000:
        q_liq_array.append(0)
        continue
        
    q_liq_array.append(q_liq.x)

In [92]:
q_inj_array

array([75000.        , 76052.63157895, 77105.26315789, 78157.89473684,
       79210.52631579, 80263.15789474, 81315.78947368, 82368.42105263,
       83421.05263158, 84473.68421053, 85526.31578947, 86578.94736842,
       87631.57894737, 88684.21052632, 89736.84210526, 90789.47368421,
       91842.10526316, 92894.73684211, 93947.36842105, 95000.        ])

In [93]:
q_liq_array

[66.38194892372964,
 66.39759944900509,
 66.41177827216481,
 66.42452646938632,
 66.43587256167076,
 66.44585866169419,
 66.45451153707312,
 66.4618581756446,
 66.46793830492012,
 66.47278125659872,
 66.47640800157313,
 66.47885815537755,
 66.48014778184464,
 66.48030063141064,
 66.47935259865342,
 66.47732046731568,
 66.47423433196946,
 66.47010851677508,
 66.46496515484733,
 66.45883417690659]

In [102]:
# Сравним ответ на 1 задание
import pytest

# Допустимый процент ошибки
rtol = 0.02

try:
    assert max(q_liq_array) == pytest.approx(output_data["t2"]["q_liq"], rtol)
    assert q_inj_array[q_liq_array.index(max(q_liq_array))] == pytest.approx(output_data["t2"]["q_inj"], abs=3000)
except AssertionError as e:
    print("Ответ студента неверный")
    print("Unifloc: ",  max(q_liq_array), q_inj_array[q_liq_array.index(max(q_liq_array))])
    print("Студент: ", output_data["t2"]["q_liq"], output_data["t2"]["q_inj"])
    
    # Если дебит сходится, а закачка немного расходится, думаю можно считать ответ правильным

Ответ студента неверный
Unifloc:  66.48030063141064 88684.21052631579
Студент:  66.6411531060521 85000
